In [2]:
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd


recrutamento = pd.read_csv('recruitment_data.csv', sep=',', decimal='.')

features = list(recrutamento.columns.values)
classificacao = 'HiringDecision'

# features.remove('PreviousCompanies')
# features.remove('Age')
# features.remove('Gender')
# features.remove('DistanceFromCompany')

#Remove o output
features.remove(classificacao)

X = recrutamento[features].to_numpy() 
Y = recrutamento[classificacao].to_numpy() 

#IMPLEMENTANDO O RANDOMFOREST SEM BALANCEAMENTO
import pandas as pd
from sklearn.model_selection import train_test_split

# Divisão da base de dados entre treinamento e teste (30% para testar e 70% para treinar)
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X,
                                                                  Y,
                                                                  test_size = 0.3,
                                                                  random_state = 22)





In [4]:
from tensorflow.keras.models import Sequential

autoencoder = Sequential([
    layers.Dense(5, activation='relu', input_shape=(X_treinamento.shape[1],)),
    layers.Dense(1, activation='relu', name='bottleneck'),
    layers.Dense(5, activation='relu'),
    layers.Dense(X_treinamento.shape[1], activation='sigmoid')
])

autoencoder.compile(optimizer='rmsprop', loss='binary_crossentropy')

# Treinar o autoencoder
autoencoder.fit(X_treinamento, X_treinamento, epochs=500, batch_size=32, validation_data=(X_teste, X_teste), verbose=0)

# Extrair as características do codificador
encoder = Sequential(autoencoder.layers[:2])
X_train_encoded = encoder.predict(X_treinamento)
X_test_encoded = encoder.predict(X_teste)



In [10]:
# Treinar um classificador com as características extraídas
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


classifier = RandomForestClassifier()
classifier.fit(X_train_encoded, y_treinamento)

# Avaliar o classificador
y_pred = classifier.predict(X_test_encoded)
accuracy = accuracy_score(y_teste, y_pred)
print(f'Acurácia: {accuracy:.2f}')

Acurácia: 0.63
